In [1]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import copy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
data_dir = '../data/Humor,Hist,Media,Food'
file_names = os.listdir(data_dir) #reading the data directory to list all the files
file_paths = [(data_dir + '/' + fname) for fname in file_names] #forming file paths
docID_to_doc_mapping = {} #forming docID to doc name mapping
for i in range(len(file_names)):
    docID_to_doc_mapping[i] = file_names[i]

In [3]:
def remove_punct(tok):
    '''
        Removing punctations from tokens
    '''
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok
def remove_space(tok):
    '''
        Removing blank space toks
    '''
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):
    '''
        This function preprocesses the file text.
        Input: file_text in string form represting the text of a file
        Returns: cleaned_toks, word tokens present in the file after preprocessing
    '''

    #converting the text to lowercase
    ftext = file_text.lower()

    #performing word tokenization
    file_toks = word_tokenize(ftext)

    #removing the stopwords from tokens
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in file_toks if tok not in stop_words]

    #removing punctuation marks from tokens
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        if(ctok != ""):
            toks_no_punct.append(ctok)
    
    #Removing blank space tokens
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        if(ctok != ""):
            cleaned_toks.append(ctok)

    return cleaned_toks

def cleanQuery(query_text):
    '''
        Preprocessing the query text
        Input: query_text, string of the phrase query text
        Returns: cleaned_toks, an array containg the preprocessed query tokens
    '''

    #We perform the same preprocessing steps on the query as we did for the file text

    #converting the text to lowercase
    qtext = query_text.lower()
    
    #performing word tokenization
    query_toks = word_tokenize(qtext)
    
    #removing the stopwords from tokens
    stop_words = list(set(stopwords.words('english')))
    query_toks = [tok for tok in query_toks if tok not in stop_words]
    
    #removing punctuation marks from tokens
    toks_no_punct = []
    for tok in query_toks:
        ctok = remove_punct(tok)
        if(ctok != ""):
            toks_no_punct.append(ctok)
    
    #Removing blank space tokens
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        if(ctok != ""):
            cleaned_toks.append(ctok)
    
    return cleaned_toks

In [4]:
def read_file(fpaths):
    '''
        Reads the files and preprocess every file's text to form word tokens for every file.
        Returns a 2-D list containing word tokens for every file
    '''
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='ignore') #open the file
        ftxt_unprocessed = f.read() #read the text of the file
        ftoks = preprocess_file(ftxt_unprocessed) #preprocessing the text to form word tokens
        file_tokens.append(ftoks)
    return file_tokens


In [5]:
def getDocsFromID(docID_to_doc, doc_IDs):
    '''
        Given a list of document IDs, it outputs the document names corresponding to thos IDs.
        Input: docID_to_docs (mapping between docID -> doc_name), docIDs - list of input document IDs
        Returns: doc_names - list of doc_names corresponding to document IDs in doc_IDs
    '''
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(docID_to_doc[doc_ID])
    return doc_names

In [6]:
document_toks = read_file(file_paths) #reading the files to collect tokens for all the documents

In [7]:
#froming the total vocabulary using the individual document tokens
vocabulary_set = set()
for doc_tok in document_toks:
    for tok in doc_tok:
        vocabulary_set.add(tok)
vocabulary_list = list(vocabulary_set)

#creating term-id and id-term mapping
id_to_term = {}
term_to_id = {}
for i in range(len(vocabulary_list)):
    id_to_term[i] = vocabulary_list[i]
    term_to_id[vocabulary_list[i]] = i

Jaccard Coefficient

In [8]:
def compute_jaccard_coeff(query_toks, doc_toks):
    '''
        This function computes the Jaccard Coefficient of a Document and a Query given the document tokens and the query tokens.
    '''
    query_tok_set = set(query_toks)
    doc_toks_set = set(doc_toks)
    num_intersection = len(list(query_tok_set & doc_toks_set)) #intersection of document_tokens and query_tokens sets
    num_union = len(list(query_tok_set | doc_toks_set)) #union of document_tokens and query_tokens sets
    jaccard = num_intersection / num_union #calculating jaccard coefficient value
    return jaccard

def perform_jaccard_scoring(query_toks, document_toks):
    '''
        Calculates the Jaccard coefficient between a query and all the documents in our dataset. 
    '''
    jaccard_coeff_values = {}
    for i in range(len(document_toks)):
        jaccard_coeff_i = compute_jaccard_coeff(query_toks, document_toks[i]) #calculate jaccard coefficient value b/w query and i_th document
        jaccard_coeff_values[i] = jaccard_coeff_i
    return jaccard_coeff_values

def get_relevant_by_jaccard(jaccard_coeff_values):
    '''
        This function gives the top 5 relevant documents (IDs and Jaccard coefficient values) based on the value of the Jaccard coefficient.
    '''
    ranked_order_by_jaccard = dict(sorted(jaccard_coeff_values.items(), key=lambda item: item[1], reverse=True)) #sort in descending order based on the value of Jaccard coefficient
    top_5_docID = list(ranked_order_by_jaccard.keys())[:5] #take top 5 document IDs from the descending sorted dictionary
    top_5_jaccard = list(ranked_order_by_jaccard.values())[:5] #take corresponding top 5 Jaccard coefficient values from the descending sorted dictionary
    return top_5_docID, top_5_jaccard

def run_jaccard(query, document_toks):
    '''
        This function is used to run the whole process.
    '''
    query_toks = cleanQuery(query) #cleaning (preprocessing and tokenizing) the query
    if(len(query_toks) == 0): #if the number of query tokens remaining after preprocessing is zero
        print("no. of query tokens after preprocessing is 0. Jaccard coefficient with all documents is equal to 0")
        for i in range(5):
            print(f"{i + 1} : {docID_to_doc_mapping[i]} (0)") 
    jaccard_scores = perform_jaccard_scoring(query_toks, document_toks) #calculate the Jaccard coefficient between a query and all the documents in our dataset
    top_5_doc_ids, top_5_jaccard_score = get_relevant_by_jaccard(jaccard_scores) #get the top 5 relevant documents (IDs and Jaccard coefficient values) based on the value of the Jaccard coefficient.
    top_5_doc_names = getDocsFromID(docID_to_doc_mapping, top_5_doc_ids) #get document names from their IDs
    print(f"Query Text = {query}")
    print(f"Query tokens after preprocessing = {query_toks}")
    print(f"Top 5 relevant documents based on the value of the Jaccard coefficient : ")
    for i in range(len(top_5_doc_names)):
        print(f"{i + 1} : {top_5_doc_names[i]} ({top_5_jaccard_score[i]})") 

TF-IDF

In [9]:
def compute_raw_term_frequency(document_toks):
    '''
        This function calculates the term frequency of each term present in a document. It returns a dictionary with key as document IDs. 
        Each key has a value equal to another dictionary of term-wise frequencies of terms present in that document.
    '''
    #Basically here we  calculate the raw count of the word in each document and stored it as a nested dictionary for each document.
    raw_term_freq = {} #dictionary to hold term_frequency dictionary for each document
    for i in range(len(document_toks)): #iterate over all the documents
        raw_term_freq[i] = {}
        unique_toks, tok_freq = np.unique(document_toks[i], return_counts=True) #calculate the unique tokens and its corresponding term frequency in the i_th document
        for j in range(len(unique_toks)):
            raw_term_freq[i][unique_toks[j]] = tok_freq[j] #fill the dictionary holding term frequencies for each document
    return raw_term_freq

def generate_term_postings(document_toks):
    '''
        This function generates the term-wise posting lists. Returns a dictionary corresponding to the posting list of all the terms 
        containg the document IDs correspondg to each term. 
    '''
    term_posting_lists = {} #dictionary to hold the posting lists. Key is the term and value is the posting list.
    for i in range(len(document_toks)): #Iterate over all the files
        for tok in document_toks[i]: #For each file, iterate over all the file tokens
            if(tok not in term_posting_lists.keys()): #if the token is not yet present as a term in the dict
                term_posting_lists[tok] = [i] #create a new entry for the term in the index and intilize it with document_ID 'i'
            else: #else if the token is already present as a term in the dict
                if(i not in term_posting_lists[tok]): #if the document ID 'i' is not yet added to the posting list for the term 
                    term_posting_lists[tok].append(i)
    return term_posting_lists

def compute_document_frequency(term_posting_lists):
    '''
        This function computes the document frequency for each term using the term posting lists. Document frequency of a term is equal
        to the no. of docs in its posting list
    '''
    term_df = {}
    for term in term_posting_lists.keys(): #iterate over all the terms
        term_df[term] = len(term_posting_lists[term]) #document frequency of a term is equal to the no. of docs in its posting list
    return term_df

def compute_IDF(term_df, num_total_docs):
    '''
        This function computes the inverse document frequency (idf) for each term given its document frequency (df)
    '''
    term_idf = {}
    for term in term_df.keys(): #iterate over all the terms
        idf_value = np.log10(num_total_docs / (term_df[term] + 1)) #Using formula IDF(word)=log(total no. of documents/document frequency(word)+1) [with 1-smoothing]
        term_idf[term] = idf_value
    return term_idf

def compute_tf_weight(scheme, term, doc_tfs):
    '''
        This function calculates the TF-weight of a term in a document given the dictionary of term frequencies of all the terms in that document.
        Input - scheme: denotes what weighting scheme to use, term- the term to calculate the TF-weight for, doc_tfs: dictionary of term frequencies of all the terms in that document
    '''

    #We have used the formulas provided in the assignment for each weighting sscheme to calculate TF-weight

    if(scheme == "binary"): #if scheme is binary
        if(term in doc_tfs.keys()): #if term is present in the document
            return 1
        else: #if term is not present in the document
            return 0
    elif(scheme == "raw_count"): #if scheme is raw_count
        if(term in doc_tfs.keys()): #if term is present in the document
            return doc_tfs[term]
        else: #if term is not present in the document
            return 0
    elif(scheme == "term_frequency"): #if scheme is term_frequency
        if(term in doc_tfs.keys()): #if term is present in the document
            total_terms = sum(doc_tfs.values())
            return doc_tfs[term] / total_terms
        else: #if term is not present in the document
            return 0
    elif(scheme == "log_normalization"): #if scheme is log_normalization
        if(term in doc_tfs.keys()): #if term is present in the document
            return np.log10(1 + doc_tfs[term])
        else: #if term is not present in the document
            return 0
    elif(scheme == "double_normalization"): #if scheme is double_normalization
        if(term in doc_tfs.keys()): #if term is present in the document
            t1 = 0.5
            t2 = (0.5)*(doc_tfs[term] / max(doc_tfs.values()))
            return t1 + t2
        else: #if term is not present in the document
            return 0.5

def generate_tf_idf_matrices(document_toks, vocabulary_list):
    '''
        This function generates the TF-IDF matrices for all the schemes mentioned. All the TF-ODF matrices are stored in the dictionary 
        'tf_idf_matrix_by_scheme'. Function returns this dictionary along with two other dictionaries raw_tfs containing the raw term frequencies 
        for all terms in all documents and also the term_idfs
    '''
    num_docs = len(list(document_toks)) #number of docs
    print(f"Num docs = {num_docs}")
    raw_tfs = compute_raw_term_frequency(document_toks) #This function calculates the raw term frequency of each term present in each document
    term_wise_postings = generate_term_postings(document_toks) #get the posting list for all the terms
    term_document_freq = compute_document_frequency(term_wise_postings) #getting the document frequency for all terms
    term_idfs = compute_IDF(term_document_freq, num_docs) #getting the idf for all terms
    num_words = len(vocabulary_list) #number of unique words across all documents (vocabulary)
    tf_idf_matrix_by_scheme = {} #dictionary to store all tf-idf matrices
    schemes_list = ['binary', 'raw_count', 'term_frequency', 'log_normalization', 'double_normalization']
    for scheme in schemes_list:
        tf_idf_matrix_by_scheme[scheme] = np.zeros((num_docs, num_words)) #initializing the matrices of size (num_docs x  num_words_in_vocabulary)
    for scheme in schemes_list: #loop to generate tf-idf matrix for each scheme
        print(f"Generating for scheme : {scheme}")
        for i in tqdm(range(num_docs)):
            for j in range(num_words):
                tf_weight = compute_tf_weight(scheme, id_to_term[j], raw_tfs[i]) #compute the TF-weight for a term in a document
                idf = term_idfs[id_to_term[j]] #getting idf of the term
                tf_idf_matrix_by_scheme[scheme][i][j] = tf_weight * idf #value in tf-idf matrix is product of wieght of tf with idf
    return raw_tfs, term_idfs, tf_idf_matrix_by_scheme

def get_query_vector(query_toks, scheme, term_idfs, vocab_len):
    '''
        This functions generates the query vector of size same as that of size of our vocabulary. Input is the query tokens and output is the query vector
    '''
    num_query_toks = len(query_toks) #number of tokens in the query
    query_vector = [0] * vocab_len #initializing a query vector of length vocab size
    query_tfs = {} #raw-tf of terms in query
    for i in range(num_query_toks):
        query_tfs[query_toks[i]] = 0
    for i in range(num_query_toks):
        query_tfs[query_toks[i]] += 1 #calculating raw-tf of terms in the query
    for i in range(len(query_vector)):
        term_tf_weight = compute_tf_weight(scheme, id_to_term[i], query_tfs) #computing the tf-weight for the terms wrt to query terms
        if(id_to_term[i] not in term_idfs.keys()): #if the term is not in idf terms, idf_val is 0
            idf_val = 0
        else:
            idf_val = term_idfs[id_to_term[i]] #else idf value is the value of idf wrt to documents we have
        query_vector[i] = term_tf_weight * idf_val #tf-idf computation for the query vector
    return query_vector

def process_tf_idf_query(query, tf_idf_matrix_dict, idf_values, vocab_len):
    '''
        This function processes a query according to TF-IDF based scheme to find the top 5 relevant documents based on the TF-IDF score for a query. 
        This is done for all weighting schemes
    '''
    query_toks = cleanQuery(query) #get query tokens after preprocessing
    print(f"Query : {query}")
    print(f"Query tokens = {query_toks}\n")
    schemes_list = ['binary', 'raw_count', 'term_frequency', 'log_normalization', 'double_normalization'] #all weighting schemes
    for scheme in schemes_list: #one by one perform this for all the weighting Sschemes
        print(f"\n---------------- Scheme : {scheme} -------------------\n") 
        query_vector = np.array(get_query_vector(query_toks, scheme, idf_values, vocab_len)).reshape((vocab_len, 1)) # getting the |v|x1 query vector where |v| is size of vocab
        tf_idf_matrix = np.array(tf_idf_matrix_dict[scheme]) # Getting the |d| x |v| tf-idf matrix for the chosen weighting scheme |d| -> no. of docs
        document_scores = np.dot(tf_idf_matrix, query_vector) #computing the scores for all documents. The dot product between tf-idf matrix and query vector returns a |d| x 1 vector with scores wrt each document for a particular query.
        document_id_to_score = {}
        for i in range(document_scores.shape[0]): #mapping document id to their scores
            document_id_to_score[i] = document_scores[i][0]
        document_id_to_score = dict(sorted(document_id_to_score.items(), key=lambda item: item[1], reverse=True)) #sort the scores in descending order
        top_5_doc_ids = list(document_id_to_score.keys())[:5] #choose the top 5 scores
        top_5_doc_names = getDocsFromID(docID_to_doc_mapping, top_5_doc_ids) #getting doc names from ID
        for i in range(5):
            print(f"{i} -> {top_5_doc_names[i]} | Score = {document_id_to_score[top_5_doc_ids[i]]}")
        print("-----------------------------------------------------------")

In [10]:
raw_tf_docs, term_idfs, tf_idf_matrix_dict  = generate_tf_idf_matrices(document_toks, vocabulary_list)
pickle.dump(tf_idf_matrix_dict, open('./tf_idf_matrices_q1.pkl', 'wb'))

Num docs = 1133
Generating for scheme : binary


100%|██████████| 1133/1133 [02:30<00:00,  7.51it/s]


Generating for scheme : raw_count


100%|██████████| 1133/1133 [02:34<00:00,  7.35it/s]


Generating for scheme : term_frequency


100%|██████████| 1133/1133 [03:15<00:00,  5.78it/s]


Generating for scheme : log_normalization


100%|██████████| 1133/1133 [02:43<00:00,  6.95it/s]


Generating for scheme : double_normalization


100%|██████████| 1133/1133 [02:44<00:00,  6.88it/s]


In [11]:
file_to_read = open("./tf_idf_matrices_q1.pkl", "rb")
tf_idf_matrix_dict = pickle.load(file_to_read)

In [12]:
def run_question1():
    method_to_run = int(input("Enter 1 -> Jaccard scoring ; 2 -> TF-IDF : "))
    if(method_to_run == 1):
        query_input_text = input("Input query text")
        if(query_input_text == ""):
            print("Empty Query !!")
            return
        else:
            run_jaccard(query_input_text, document_toks)
    else:
        query_input_text = input("Input query text")
        if(query_input_text == ""):
            print("Empty Query !!")
            return
        else:
            process_tf_idf_query(query_input_text, tf_idf_matrix_dict, term_idfs, len(vocabulary_list))


In [ ]:
run_question1()

In [43]:
# input_query = input()
# process_tf_idf_query(input_query, tf_idf_matrix_dict, term_idfs, len(vocabulary_list))

Query : good
Query tokens = ['good']


---------------- Scheme : binary -------------------

0 -> 1st_aid.txt | Score = 0.10409730823826448
1 -> abbott.txt | Score = 0.10409730823826448
2 -> acronyms.txt | Score = 0.10409730823826448
3 -> ads.txt | Score = 0.10409730823826448
4 -> adt_miam.txt | Score = 0.10409730823826448
-----------------------------------------------------------

---------------- Scheme : raw_count -------------------

0 -> manners.txt | Score = 8.848271200252482
1 -> practica.txt | Score = 5.621254644866283
2 -> oldtime.sng | Score = 4.684378870721901
3 -> mlverb.hum | Score = 4.059795021292315
4 -> vegan.rcp | Score = 3.8516004048157857
-----------------------------------------------------------

---------------- Scheme : term_frequency -------------------

0 -> oldtime.sng | Score = 0.007173627673387291
1 -> f_tang.txt | Score = 0.004053346515472245
2 -> popmach | Score = 0.0033579776851053057
3 -> bless.bc | Score = 0.0023658479145060108
4 -> beer.hum | Score = 